In [2]:
import json
import pandas as pd
import glob


In [9]:
len(glob.glob("/home/gujiashe/Tk-Instruct/data/tasks/*.json"))
# 😧🙈🙂😘😡😩🤬😂🤷 🏻🙄😔😮👊🇺🇸👍😈😎💪🤣😀😍😬 😊💃
# 

1613

In [25]:
tasks = {}


tasks['Categories'] = []
tasks['name'] = []
for i in range(1, 11):
    tasks['Example_' + str(i) + '_input'] = []
    tasks['Example_' + str(i) + '_output'] = []
    tasks['Example_' + str(i) + '_explanation'] = []
paths = []

for task_path in glob.glob("/home/gujiashe/Tk-Instruct/data/tasks/*.json"):
    with open(task_path, "r") as task_f:

        
        s = task_f.read()
        task = json.loads(s)
        if len(task["Input_language"])>1 or task["Input_language"][0] != "English":
            continue
        if len(task["Output_language"])>1 or task["Output_language"][0] != "English":
            continue

        i = 1
        for example in task['Positive Examples']:
            if i>10:
                break
            tasks['Example_' + str(i) + '_input'].append(example['input'])
            tasks['Example_' + str(i) + '_output'].append(example['output'])
            tasks['Example_' + str(i) + '_explanation'].append('Positive. ' + example['explanation'])
            i+=1
            


        for example in task['Negative Examples']:
            if i>10:
                break
            tasks['Example_' + str(i) + '_input'].append(example['input'])
            tasks['Example_' + str(i) + '_output'].append(example['output'])
            tasks['Example_' + str(i) + '_explanation'].append('Negative. ' + example['explanation'])
            i+=1
            
        
        for example in task['Instances']:
            if i>10:
                break
            tasks['Example_' + str(i) + '_input'].append(example['input'])
            tasks['Example_' + str(i) + '_output'].append(example['output'][0])
            tasks['Example_' + str(i) + '_explanation'].append("")
            i+=1

        Categories = task['Categories'][0]
        name = task_path.split("/")[-1].split(".")[0]
        tasks['name'].append(name)
        tasks['Categories'].append(Categories)
        paths.append(task_path)
        # break
tasks = pd.DataFrame(tasks)

In [7]:
tasks.to_csv("tasks.csv", index=None)

In [15]:
tasks = pd.read_csv("tasks copy.csv")

In [19]:
print(tasks.iloc[34]['Examples'])

Positive Examples: 

input: 

What is the sentiment of the following sentence towards the entity Bill Clinton ? Bill Clinton knows how to win friends and influence people.

output: 

Positive

explanation: 

Here the author of the document praises Bill for this ability to win friends.



input: 

What is the sentiment of the following document towards the entity Bill Clinton ? Former president Bill Clinton is scheduled to visit Sollers Point Library in Dundalk on Monday during a trip to Baltimore. 

output: 

Neutral

explanation: 

It is just reporting some facts and it is not clear about the sentiment of the author towards the fact.



input: 

What is the sentiment of the following document towards the entity Bill Clinton ? But the bill that President Clinton signed is not welfare reform. 

output: 

Negative

explanation: 

Here the author raises some concerns about the bill that Clinton signed on is not for welfare reform.



Negative Examples: 

input: 

What is the sentiment of 

In [ ]:
tasks = {}


tasks['Categories'] = []
tasks['pos_input'] = []
tasks['pos_output'] = []
tasks['pos_explanation'] = []

tasks['neg_input'] = []
tasks['neg_output'] = []
tasks['neg_explanation'] = []
paths = []

for task_path in glob.glob("/home/gujiashe/Tk-Instruct/data/tasks/*.json"):
    with open(task_path, "r") as task_f:
        s = task_f.read()
        task = json.loads(s)
        if len(task["Input_language"])>1 or task["Input_language"][0] != "English":
            continue
        if len(task["Output_language"])>1 or task["Output_language"][0] != "English":
            continue

        if len(task['Positive Examples']) == 0:
            tasks['pos_input'].append("")
            tasks['pos_output'].append("")
            tasks['pos_explanation'].append("")
        
        if len(task['Negative Examples']) == 0:
            tasks['neg_input'].append("")
            tasks['neg_output'].append("")
            tasks['neg_explanation'].append("")
        for example in task['Positive Examples']:
            tasks['pos_input'].append(example['input'])
            tasks['pos_output'].append(example['output'])
            tasks['pos_explanation'].append(example['explanation'])
            break


        for example in task['Negative Examples']:
            tasks['neg_input'].append(example['input'])
            tasks['neg_output'].append(example['output'])
            tasks['neg_explanation'].append(example['explanation'])
            break

        Categories = task['Categories'][0]
        tasks['Categories'].append(Categories)
        paths.append(task_path)
tasks = pd.DataFrame(tasks)